In [ ]:
from .population_cmaes import PopulationCMAES

def cmaes(problem, N, T, l=1, treshold=0.44):
    P = PopulationCMAES(l, treshold)
    P.initialize(problem, N)
    for t in range(T):
        children = []
        for individual in P.individuals:
            child = individual.clone()
            child.mutate()
            children.append(child)
        P.append_many(children)
        P.fastNondominatedSort()
        P.crowdingDistanceSort()
        for individual in P.individuals:
            if individual.parent is None:
                pass
            else:
                lambda_succ = int(individual.crowdedComparison(individual.parent)) / P.l
                individual.parent.update_step_size(lambda_succ, P.target_success_rate, P.delta, P.l)
                individual.update_step_size(lambda_succ, P.target_success_rate, P.delta, P.l)
                step = (individual.coordinates - individual.parent.coordinates) / individual.parent.sigma
                individual.update_covariance(step, P.treshold, P.evolution_rate, P.covariance_rate)
                individual.set_parent(None)
        Pnew = PopulationCMAES(l, treshold)
        Pnew.inherit_attributes(P)
        Pnew.elitist_selection(P, N)
        P = Pnew
    return P
